In [19]:
%env HADOOP_HOME /usr/lib/hadoop
%env HADOOP_CONF_DIR /etc/hadoop/conf
%env HADOOP_YARN_HOME /usr/lib/hadoop
%env HADOOP_MAPRED_HOME /usr/lib/hadoop
%env HADOOP_HDFS_HOME /usr/lib/hadoop-hdfs

%env SPARK_HOME /usr/lib/spark
%env SPARK_CONF_DIR /etc/spark/conf
#%env PYTHONPATH /opt/spark/python/lib/py4j-0.10.7-src.zip
%env PYSPARK_PYTHON /opt/conda/bin/python
%env PYSPARK_DRIVER_PYTHON /opt/conda/bin/python
%env PYTHONPATH /usr/lib/spark/python/lib/py4j-0.10.7-src.zip
#%env PATH /usr/lib/hadoop/bin:/bin:/usr/lib/spark/bin::/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/tmp:/opt/oracle/instantclient_12_1:/home/ubuntu/.local/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/share/scala/bin:/opt/oracle/instantclient_12_1:/home/ubuntu/.local/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/hadoop-hdfs/bin:/opt/hadoop/bin
!hdfs dfs -rm -r /user/dominospark/example*
!echo $PYTHONPATH

env: HADOOP_HOME=/usr/lib/hadoop
env: HADOOP_CONF_DIR=/etc/hadoop/conf
env: HADOOP_YARN_HOME=/usr/lib/hadoop
env: HADOOP_MAPRED_HOME=/usr/lib/hadoop
env: HADOOP_HDFS_HOME=/usr/lib/hadoop-hdfs
env: SPARK_HOME=/usr/lib/spark
env: SPARK_CONF_DIR=/etc/spark/conf
env: PYSPARK_PYTHON=/opt/conda/bin/python
env: PYSPARK_DRIVER_PYTHON=/opt/conda/bin/python
env: PYTHONPATH=/usr/lib/spark/python/lib/py4j-0.10.7-src.zip
2021-08-13 13:39:43,747 WARN ipc.Client: Exception encountered while connecting to the server : org.apache.hadoop.security.AccessControlException: Client cannot authenticate via:[TOKEN, KERBEROS]
rm: DestHost:destPort ip-10-0-1-76.us-west-2.compute.internal:8020 , LocalHost:localPort run-611593d6b7181f12fed2a4fc-5z79j/10.0.51.132:0. Failed on local exception: java.io.IOException: org.apache.hadoop.security.AccessControlException: Client cannot authenticate via:[TOKEN, KERBEROS]
/usr/lib/spark/python/lib/py4j-0.10.7-src.zip


In [11]:
#https://saagie.zendesk.com/hc/en-us/articles/360029759552-PySpark-Read-and-Write-Files-from-HDFS
#ON EMR FETCH /usr/lib/hadoop-lzo/lib/
#spark.executor.extraClassPath /etc/hadoop/conf:/etc/hive/conf:/usr/lib/hadoop-lzo/lib/*:/usr/share/aws/aws-java-sdk/*:/usr/share/aws/emr/emrfs/conf:/usr/share/aws/emr/emrfs/lib/*:/usr/share/aws/emr/emrfs/auxlib/*

#spark.driver.extraClassPath /etc/hadoop/conf:/etc/hive/conf:/usr/lib/hadoop-lzo/lib/*:/usr/share/aws/aws-java-sdk/*:/usr/share/aws/emr/emrfs/conf:/usr/share/aws/emr/emrfs/lib/*:/usr/share/aws/emr/emrfs/auxlib/*
#%env SUBMIT_ARGS '--jars /opt/spark/conf/hadoop-lzo-0.4.19.jar'
from pyspark.sql import SparkSession


In [12]:
sparkSession = SparkSession.builder.appName("example-pyspark-read-and-write") \
.master('local[*]') \
.config("fs.default.name", "hdfs://10.0.1.76:8020") \
.getOrCreate()


In [13]:
sparkSession

In [14]:
import random
#Create datasets
data = []
cnt=10000
for i in range(cnt):
   data.append((i,random.randint(1,100),random.randint(1,100),random.randint(1,100)))

In [15]:
hdfs_path='/user/dominospark/example'
df = sparkSession.createDataFrame(data)

# Write into HDFS
df.write.csv(hdfs_path)

In [16]:
df_load = sparkSession.read.csv(hdfs_path)
df_load.show()

+---+---+---+---+
|_c0|_c1|_c2|_c3|
+---+---+---+---+
|  0|  3| 48| 65|
|  1| 48| 57|  6|
|  2| 82| 40| 11|
|  3| 25| 14| 42|
|  4| 18| 74| 79|
|  5| 94| 81| 12|
|  6| 14| 36| 98|
|  7| 19| 58| 37|
|  8|  6| 89| 93|
|  9| 49|  7| 78|
| 10| 64| 65| 57|
| 11| 91| 30| 40|
| 12|  5| 10| 73|
| 13| 16| 59|  2|
| 14| 38| 88| 96|
| 15| 55| 99| 84|
| 16| 24| 61| 68|
| 17| 50| 50| 86|
| 18| 17| 85| 25|
| 19| 80| 83| 25|
+---+---+---+---+
only showing top 20 rows



In [17]:
sparkSession.stop()

In [18]:
!hdfs dfs -ls /user/dominospark

Found 4 items
drwxr-xr-x   - dominospark dominospark          0 2021-08-12 22:05 /user/dominospark/.sparkStaging
drwxr-xr-x   - dominospark dominospark          0 2021-08-12 22:06 /user/dominospark/example
drwxr-xr-x   - dominospark dominospark          0 2021-08-12 21:36 /user/dominospark/mypi
drwxr-xr-x   - dominospark dominospark          0 2021-08-11 15:36 /user/dominospark/test
